In [51]:
import os
import pandas as pd
from pandas.core.frame import DataFrame

class ShebaoSalaryTable:
    def __init__(self,fn_sb,fn_slry):
        self.fn_sb=os.path.join(os.getcwd(),fn_sb)
        self.fn_slry=os.path.join(os.getcwd(),fn_slry)
        
    def WriteToSalary(self):
        df_sb=pd.read_csv(self.fn_sb)
        df_slry=pd.read_excel(self.fn_slry,skiprows=4,usecols=('c:g'))
        df_slry.rename(columns={'Unnamed: 2':'姓名','Unnamed: 3':'预发工资'},inplace=True)
        for i in range(2):
            df_slry.drop([len(df_slry)-1],inplace=True)
            
        nameList=df_slry['姓名'].values.tolist()
        
#         nameList=['骆莹莹']
        items=['职工基本养老保险','职工基本医疗保险','失业保险']
        
        output=[]
        for name in nameList:
            eachPerson=[name]
            for item in items:        
                YingJiao=df_sb.loc[(df_sb['姓名']==name)& (df_sb['缴费对象']=='个人') & (df_sb['险种类型']==item)]
                eachPerson.append(YingJiao['本期应缴'].values.tolist()[0])
            output.append(eachPerson)
        print(output)
        
        toWriteList=DataFrame(output)
        toto=toWriteList.drop(0,axis=1)
        
        toto.to_excel(self.fn_slry,startrow=6,startcol=5)
        
#         print(toto)
        
        print('完成')
        
        
if __name__=='__main__':
    sb=ShebaoSalaryTable('7月社保数据.csv','薪酬表.xlsx')
    sb.WriteToSalary()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb8 in position 0: invalid start byte